<a href="https://colab.research.google.com/github/mostafa-ja/sample/blob/master/PyTorch_Tutorial_16_How_To_Use_The_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Youtube link](https://www.youtube.com/watch?v=VJW9wU-1n18&list=PLqnslRFeH2UrcDBWF5mfPGpqQDSta6VK4&index=16)

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [3]:
writer = SummaryWriter('runs/mnist1')

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# hyper parameters
input_size = 784
hidden_size = 500
num_classes = 10
batch_size = 4
num_epochs = 1
learning_rate = 0.001

# Load data

In [6]:
data_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                 torchvision.transforms.Normalize([0.1307,], [0.3081,])]) # just one element because of shape (1, 28, 28)

In [7]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST('/content/data',train=True, transform=data_transform, download=True)
test_dataset = torchvision.datasets.MNIST('/content/data',train=False, transform=data_transform, download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False)

In [8]:
# samples

example_data,example_label = next(iter(train_loader))
img_grid = torchvision.utils.make_grid(example_data)


############## TENSORBOARD ########################
writer.add_image('samples',img_grid)

# writer.close()
# %tensorboard --logdir=runs

In [9]:
print(example_data.shape)
print(img_grid.shape)

torch.Size([4, 1, 28, 28])
torch.Size([3, 32, 122])


In [10]:
# Fully connected neural network with one hidden layer

class NeuralNet(nn.Module):

  def __init__(self,input_size,hidden_size,num_classes):
    super().__init__()
    self.linear1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size,num_classes)

  def forward(self,x):
    out = self.linear1(x)
    out = self.relu(out)
    out = self.linear2(out)
    # no activation and no softmax at the end
    return out

In [11]:
model = NeuralNet(input_size,hidden_size,num_classes).to(device)   # Set the model to run on the GPU.


############## TENSORBOARD ########################
writer.add_graph(model,example_data.reshape(-1,28*28).to(device))  # ATTENTION : must do .to(device) for example_data

# writer.close()
# %tensorboard --logdir=runs


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

# Training the model

In [15]:
running_loss = 0.0
running_correct = 0
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  print(f'Epoch {epoch+1}/{num_epochs} ')
  print('_ _ ' * 10)

  for i,(images,labels) in enumerate(train_loader):
    # origin shape: [batch_s, 1, 28, 28] , resized: [100, 784]
    images = images.reshape(-1,28*28).to(device)
    labels = labels.to(device) # # Set the data to run on the GPU.

    outputs = model(images)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()     # OR running_loss += loss.item()* images.size(0)   :  images.size(0) == images.shape[0] == batch_size

    _, predicted = torch.max(outputs.data, 1)                   # ATTENTION: output.data is like loss.item() remove grad graph from data,
                                                                # but we cant use item() beacuse the result is not one element tensor
    running_correct += (labels==predicted).sum().item()

    if (i+1) % 100 == 0:
      print(f'Step {i+1}/{n_total_steps} , Loss : {loss.item():.4f}  ')

      ############## TENSORBOARD ########################
      writer.add_scalar(tag='training loss', scalar_value= running_loss/100, global_step= n_total_steps * epoch + i)
      running_accuracy = (running_correct / 100) / predicted.size(0)  # predicted.size(0) == predicted.shape[0]
      writer.add_scalar('accuracy', running_accuracy, global_step= n_total_steps * epoch + i)
      running_loss = 0.0
      running_correct = 0
      ###################################################
    

Epoch 1/1 
__________
Step 100/15000 , Loss : 0.6008  
Step 200/15000 , Loss : 0.5181  
Step 300/15000 , Loss : 0.1389  
Step 400/15000 , Loss : 0.1191  
Step 500/15000 , Loss : 0.1029  
Step 600/15000 , Loss : 0.0095  
Step 700/15000 , Loss : 0.8239  
Step 800/15000 , Loss : 0.5510  
Step 900/15000 , Loss : 0.0842  
Step 1000/15000 , Loss : 0.8247  
Step 1100/15000 , Loss : 0.3234  
Step 1200/15000 , Loss : 0.0070  
Step 1300/15000 , Loss : 0.0019  
Step 1400/15000 , Loss : 0.1068  
Step 1500/15000 , Loss : 0.3971  
Step 1600/15000 , Loss : 0.3709  
Step 1700/15000 , Loss : 0.0131  
Step 1800/15000 , Loss : 0.0693  
Step 1900/15000 , Loss : 0.1984  
Step 2000/15000 , Loss : 0.0167  
Step 2100/15000 , Loss : 0.0883  
Step 2200/15000 , Loss : 0.0158  
Step 2300/15000 , Loss : 0.0005  
Step 2400/15000 , Loss : 0.0642  
Step 2500/15000 , Loss : 0.0785  
Step 2600/15000 , Loss : 0.0123  
Step 2700/15000 , Loss : 0.4944  
Step 2800/15000 , Loss : 0.0345  
Step 2900/15000 , Loss : 1.3620  
S

# Evaluate

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)

class_labels = []
class_preds = []
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images,labels in (test_loader):
    images = images.reshape(-1,28*28).to(device)
    labels = labels.to(device)
    outputs = model(images)
    # max returns (value ,index)
    value, index = torch.max(outputs.data,1)  # outputs.data  : .data remove grad graph, and its for more than one element data(unlike .item())
    
